# 微调大模型

### 1 什么是 SFT

预训练是 LLM 强大能力的根本来源，事实上，LLM 所覆盖的海量知识基本都是源于预训练语料。LLM 的性能本身，核心也在于预训练的工作。但是，预训练赋予了 LLM 能力，却还需要第二步将其激发出来。经过预训练的 LLM 好像一个博览群书但又不求甚解的书生，对什么样的偏怪问题，都可以流畅地接出下文，但他偏偏又不知道问题本身的含义，只会“死板背书”。这一现象的本质是因为，LLM 的预训练任务就是经典的 CLM，也就是训练其预测下一个 token 的能力，在没有进一步微调之前，其无法与其他下游任务或是用户指令适配。

因此，我们还需要第二步来教这个博览群书的学生如何去使用它的知识，也就是 SFT（Supervised Fine-Tuning，有监督微调）。所谓有监督微调，其实就是我们在第三章中讲过的预训练-微调中的微调，稍有区别的是，对于能力有限的传统预训练模型，我们需要针对每一个下游任务单独对其进行微调以训练模型在该任务上的表现。例如要解决文本分类问题，需要对 BERT 进行文本分类的微调；要解决实体识别的问题，就需要进行实体识别任务的微调。

而面对能力强大的 LLM，我们往往不再是在指定下游任务上构造有监督数据进行微调，而是选择训练模型的“通用指令遵循能力”，也就是一般通过`指令微调`的方式来进行 SFT。

所谓指令微调，即我们训练的输入是各种类型的用户指令，而需要模型拟合的输出则是我们希望模型在收到该指令后做出的回复。例如，我们的一条训练样本可以是：

    input:告诉我今天的天气预报？
    output:根据天气预报，今天天气是晴转多云，最高温度26摄氏度，最低温度9摄氏度，昼夜温差大，请注意保暖哦

也就是说，SFT 的主要目标是让模型从多种类型、多种风格的指令中获得泛化的指令遵循能力，也就是能够理解并回复用户的指令。因此，类似于 Pretrain，SFT 的数据质量和数据配比也是决定模型指令遵循能力的重要因素。

首先是指令数据量及覆盖范围。为了使 LLM 能够获得泛化的指令遵循能力，即能够在未训练的指令上表现良好，需要收集大量类别各异的用户指令和对应回复对 LLM 进行训练。一般来说，在单个任务上 500~1000 的训练样本就可以获得不错的微调效果。但是，为了让 LLM 获得泛化的指令遵循能力，在多种任务指令上表现良好，需要在训练数据集中覆盖多种类型的任务指令，同时也需要相对较大的训练数据量，表现良好的开源 LLM SFT 数据量一般在数 B token 左右。

一般 SFT 所使用的指令数据集包括以下三个键：

```json
{
    "instruction":"即输入的用户指令",
    "input":"执行该指令可能需要的补充输入，没有则置空",
    "output":"即模型应该给出的回复"
}
```

例如，如果我们的指令是将目标文本“今天天气真好”翻译成英文，那么该条样本可以构建成如下形式：

```json
{
    "instruction":"将下列文本翻译成英文：",
    "input":"今天天气真好",
    "output":"Today is a nice day！"
}
```

同时，为使模型能够学习到和预训练不同的范式，在 SFT 的过程中，往往会针对性设置特定格式。例如，LLaMA 的 SFT 格式为：

    ### Instruction:\n{{content}}\n\n### Response:\n

其中的 content 即为具体的用户指令，也就是说，对于每一个用户指令，将会嵌入到上文的 content 部分，这里的用户指令不仅指上例中的 “instruction”，而是指令和输入的拼接，即模型可以执行的一条完整指令。例如，针对上例，LLaMA 获得的输入应该是：

    ### Instruction:\n将下列文本翻译成英文：今天天气真好\n\n### Response:\n

其需要拟合的输出则是：

    ### Instruction:\n将下列文本翻译成英文：今天天气真好\n\n### Response:\nToday is a nice day！

注意，因为指令微调本质上仍然是对模型进行 CLM 训练，只不过要求模型对指令进行理解和回复而不是简单地预测下一个 token，所以模型预测的结果不仅是 output，而应该是 input + output，只不过 input 部分不参与 loss 的计算，但回复指令本身还是以预测下一个 token 的形式来实现的。


### 1.1 高效微调-LoRA 

### 1.1.1 什么是高效微调

针对全量微调的昂贵问题，目前主要有两种解决方案：

**Adapt Tuning**。即在模型中添加 Adapter 层，在微调时冻结原参数，仅更新 Adapter 层。

具体而言，其在预训练模型每层中插入用于下游任务的参数，即 Adapter 模块，在微调时冻结模型主体，仅训练特定于任务的参数，如下图所示。

<div align='center'>
    <img src="https://raw.githubusercontent.com/datawhalechina/happy-llm/main/docs/images/6-images/3-1.png" alt="alt text" width="90%">
    <p>图6.8 Adapt Tuning</p>
</div>

每个 Adapter 模块由两个前馈子层组成，第一个前馈子层将 Transformer 块的输出作为输入，将原始输入维度 $d$ 投影到 $m$，通过控制 $m$ 的大小来限制 Adapter 模块的参数量，通常情况下 $m << d$。在输出阶段，通过第二个前馈子层还原输入维度，将 $m$ 重新投影到 $d$，作为 Adapter 模块的输出(如上图右侧结构)。

LoRA 事实上就是一种改进的 Adapt Tuning 方法。但 Adapt Tuning 方法存在推理延迟问题，由于增加了额外参数和额外计算量，导致微调之后的模型计算速度相较原预训练模型更慢。

**Prefix Tuning**。该种方法固定预训练 LM，为 LM 添加可训练，任务特定的前缀，这样就可以为不同任务保存不同的前缀，微调成本也小。具体而言，在每一个输入 token 前构造一段与下游任务相关的 virtual tokens 作为 prefix，在微调时只更新 prefix 部分的参数，而其他参数冻结不变。

也是目前常用的微量微调方法的 Ptuning，其实就是 Prefix Tuning 的一种改进。但 Prefix Tuning 也存在固定的缺陷：模型可用序列长度减少。由于加入了 virtual tokens，占用了可用序列长度，因此越高的微调质量，模型可用序列长度就越低。

目前，最主流的高效微调方法是 LoRA 微调。LoRA 微调的基本假设是，如果一个大模型是将数据映射到高维空间进行处理，这里假定在处理一个细分的小任务时，是不需要那么复杂的大模型的，可能只需要在某个子空间范围内就可以解决，那么也就不需要对全量参数进行优化了，我们可以定义当对某个子空间参数进行优化时，能够达到全量参数优化的性能的一定水平（如90%精度）时，那么这个子空间参数矩阵的秩就可以称为对应当前待解决问题的本征秩（intrinsic rank）。

预训练模型本身就隐式地降低了本征秩，当针对特定任务进行微调后，模型中权重矩阵其实具有更低的本征秩（intrinsic rank）。同时，越简单的下游任务，对应的本征秩越低。（[Intrinsic Dimensionality Explains the Effectiveness of Language Model Fine-Tuning](https://arxiv.org/abs/2012.13255)）因此，权重更新的那部分参数矩阵尽管随机投影到较小的子空间，仍然可以有效的学习，可以理解为针对特定的下游任务这些权重矩阵就不要求满秩。我们可以通过优化密集层在适应过程中变化的秩分解矩阵来间接训练神经网络中的一些密集层，从而实现仅优化密集层的秩分解矩阵来达到微调效果。

例如，假设预训练参数为 $\theta^D_0$，在特定下游任务上密集层权重参数矩阵对应的本征秩为 $\theta^d$，对应特定下游任务微调参数为 $\theta^D$，那么有：

$$\theta^D = \theta^D_0 + \theta^d M$$

这个 $M$ 即为 LoRA 优化的秩分解矩阵。

想对于其他高效微调方法，LoRA 存在以下优势：

1. 可以针对不同的下游任务构建小型 LoRA 模块，从而在共享预训练模型参数基础上有效地切换下游任务。
2. LoRA 使用自适应优化器（Adaptive Optimizer），不需要计算梯度或维护大多数参数的优化器状态，训练更有效、硬件门槛更低。
3. LoRA 使用简单的线性设计，在部署时将可训练矩阵与冻结权重合并，不存在推理延迟。
4. LoRA 与其他方法正交，可以组合。

因此，LoRA 成为目前高效微调 LLM 的主流方法，尤其是对于资源受限、有监督训练数据受限的情况下，LoRA 微调往往会成为 LLM 微调的首选方法。

### 1.1.2 如何进行 LoRA 微调

我们使用 peft 库来高效、便捷地实现 LoRA 微调。

In [7]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

file_path="formatted_finetune_data.jsonl"
df = pd.read_json(file_path, lines=True, encoding='utf-8') # 注意修改
ds = Dataset.from_pandas(df)

In [8]:
# 加载模型 tokenizer 
tokenizer = AutoTokenizer.from_pretrained('model/Qwen/Qwen3-4B-Instruct-2507', trust_remote=True)

# 打印一下 chat template
messages = [
    {"role": "system", "content": "===system_message_test==="},
    {"role": "user", "content": "===user_message_test==="},
    {"role": "assistant", "content": "===assistant_message_test==="},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
print(text)

<|im_start|>system
===system_message_test===<|im_end|>
<|im_start|>user
===user_message_test===<|im_end|>
<|im_start|>assistant
===assistant_message_test===<|im_end|>
<|im_start|>assistant



In [11]:
def preprocess_multi_turn_qwen(example):

    MAX_LENGTH = 1024
    input_ids, attention_mask, labels = [], [], []

    # 1. 提取对话内容
    # 假设每条数据都是从 system 开始，然后 human/gpt 交替
    convs = example['conversations']

    # 2. 构造符合 Qwen3 ChatML 格式的输入
    # Qwen3 依然遵循 <|im_start|> 这种 Prompt 格式
    # instruction_text = ""

    for msg in convs:
        role = msg["from"]
        content = msg["value"]

        if role == "human":
            role = "user"
        elif role == "gpt":
            role = "assistant"
        # 1. 构造角色前缀: <|im_start|>system\n 或 <|im_start|>user\n 等
        prefix = f"<|im_start|>{role}\n"
        prefix_ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]

        # 2. 构造内容: {content}
        content_ids = tokenizer(content, add_special_tokens=False)["input_ids"]

        # 3. 构造后缀: <|im_end|>\n
        suffix = "<|im_end|>\n"
        suffix_ids = tokenizer(suffix, add_special_tokens=False)["input_ids"]

        # 拼接当前轮次的完整 ID
        current_turn_ids = prefix_ids + content_ids + suffix_ids
        input_ids.extend(current_turn_ids)

        # 设置 Labels: 只有角色为 assistant 时，content 和 suffix 部分才计算 Loss
        if role == "assistant":
            # 前缀部分不学 (-100)
            turn_labels = [-100] * len(prefix_ids)
            # 内容和结束符要学 (content_ids + suffix_ids)
            turn_labels += content_ids + suffix_ids
            labels.extend(turn_labels)
        else:
            # system 和 user 的所有部分都不学 (-100)
            labels.extend([-100] * len(current_turn_ids))

    # 统一截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    attention_mask = [1] * len(input_ids)

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask
    }

In [15]:
# 使用上文定义的函数对数据集进行处理
tokenized_id = ds.map(preprocess_multi_turn_qwen, remove_columns=ds.column_names)
tokenized_id
tokenizer.decode(tokenized_id[0]["input_ids"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

'<|im_start|>system\n你是一个智能车载助手。当前用户信息: {\'Age\': 80, \'Technological_Proficiency\': \'middle\'}。 用户的长期偏好包括: Favorite Cuisine: Italian, Preferred type of Charging when being at everyday points (f.e. work, grocery, restaurant): AC, Charging Station Amenities: Wi-Fi availability, Priority for Shortest Time or Shortest Distance: Shortest Distance, Traffic Information Source Preferences: NavFlow Updates, Preferred Parking Type: On-street, Preference for Parking with Security: Yes, Fan Speed Preferences: High, Favorite Songs: Asphalt Anthems by Gritty Lyricist (Rap), Favorite Podcast Genres: Health。请基于这些信息提供个性化服务。<|im_end|>\n<|im_start|>user\n最近我一直特别想吃正宗的意大利菜，你能推荐附近不错的意大利餐厅吗？<|im_end|>\n<|im_start|>assistant\n没问题，附近4公里处有家评分很高的意大利餐厅La Cucina。要我为您导航过去吗？<|im_end|>\n<|im_start|>user\n"La Cucina"听起来很不错，请开启导航。<|im_end|>\n<|im_start|>assistant\n导航前往La Cucina餐厅的路线已启动。预计约12分钟后到达。祝您用餐愉快！<|im_end|>\n'

In [16]:
# 首先配置 LoRA 参数
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # 任务类型为 CLM，即 SFT 任务的类型
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # 目标模块，即需要进行 LoRA 微调的模块
    inference_mode=False, # 训练模式
    r=2, # Lora 秩，即 LoRA 微调的维度
    lora_alpha=8, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=2, target_modules={'up_proj', 'o_proj', 'v_proj', 'gate_proj', 'k_proj', 'q_proj', 'down_proj'}, exclude_modules=None, lora_alpha=8, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [17]:
import torch
# 重新加载一个 Base 模型
model = AutoModelForCausalLM.from_pretrained('model/Qwen/Qwen3-4B-Instruct-2507', device_map="auto",torch_dtype=torch.bfloat16)
model.enable_input_require_grads()
# 通过下列代码即可向模型中添加 LoRA 模块
model = get_peft_model(model, config)
config

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='model/Qwen/Qwen3-4B-Instruct-2507', revision=None, inference_mode=False, r=2, target_modules={'up_proj', 'o_proj', 'v_proj', 'gate_proj', 'k_proj', 'q_proj', 'down_proj'}, exclude_modules=None, lora_alpha=8, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [18]:
# 查看 lora 微调的模型参数
model.print_trainable_parameters()

trainable params: 4,128,768 || all params: 4,026,596,864 || trainable%: 0.1025


In [19]:
from swanlab.integration.transformers import SwanLabCallback

# 配置训练参数
args = TrainingArguments(
    output_dir="./output/Qwen3_4B_lora", # 输出目录
    per_device_train_batch_size=8, # 每个设备上的训练批量大小
    gradient_accumulation_steps=4, # 梯度累积步数
    logging_steps=10, # 每10步打印一次日志
    num_train_epochs=2, # 训练轮数
    save_steps=100, # 每100步保存一次模型
    learning_rate=1e-4, # 学习率
    save_on_each_node=True, # 是否在每个节点上保存模型
    gradient_checkpointing=True, # 是否使用梯度检查点
    report_to="none", # 不使用任何报告工具
)
swanlab_callback = SwanLabCallback(
    project="Qwen3-4B-lora", 
    experiment_name="Qwen3-4B-experiment"
)

In [20]:
# 然后同样使用 trainer 训练即可
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback]
)

In [21]:
# 开始训练
trainer.train()

Output()

swanlab: swanlab version 0.7.8 is available!  Upgrade: `pip install -U swanlab`

Output()

swanlab: Tracking run with swanlab version 0.6.8

swanlab: Run data will be saved locally in /root/autodl-tmp/swanlog/run-20260209_001335-oblkj2azancnis3tsa0tp

...

swanlab: 🏠 View project at https://swanlab.cn/@xiaomingchen/Qwen3-4B-lora

swanlab: 🚀 View run at https://swanlab.cn/@xiaomingchen/Qwen3-4B-lora/runs/oblkj2azancnis3tsa0tp

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.894000
20,2.317100
30,2.126000
40,1.965400
50,1.937200
60,1.895500


TrainOutput(global_step=64, training_loss=2.3296501711010933, metrics={'train_runtime': 377.3342, 'train_samples_per_second': 5.3, 'train_steps_per_second': 0.17, 'total_flos': 1.863036602285261e+16, 'train_loss': 2.3296501711010933, 'epoch': 2.0})

LoRA 微调仅保存微调后的 LoRA 参数，因此推理微调模型需要加载 LoRA 参数并合并：

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = '/root/autodl-tmp/model/Qwen/Qwen3-4B-Instruct-2507'# 基座模型参数路径
lora_path = './output/Qwen3_4B_lora/checkpoint-64' # 这里改成你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, rust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
prompt = "你是谁？"
inputs = tokenizer.apply_chat_template(
                                    [{"role": "user", "content": "开始播放新闻电台"}],
                                    add_generation_prompt=True,
                                    tokenize=True,
                                    return_tensors="pt",
                                    return_dict=True,
                                    enable_thinking=False
                                )

device = next(model.parameters()).device
inputs = {k: v.to(device) for k, v in inputs.items()}
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

我无法直接播放电台广播。不过我可以为您播报最新的新闻摘要，或者提供您感兴趣的新闻资讯。需要我为您介绍当前的热点新闻吗？
